# Projekt: System rekomendujący

<p style="text-align: right"> <i> Alicja Kołodziejska, Piotr Maślankowski </i> </p>

Celem projektu jest stworzenie i przetestowanie systemu rekomendującego na podstawie zbioru danych <b>MovieLens</b>. Zawiera on oceny, jakie wystawili filmom użytkownicy portalu internetowego. Zadaniem systemu rekomendującego jest jak najlepsze przewidzenie oceny, jaką dany użytkownik wystawiłby danemu filmowi, gdyby go obejrzał.

W projekcie badamy kilka podejść do tego problemu: rekomendację <i>user-user</i> oraz metody wykorzystujące elementy algebry liniowej.

## Opis zestawu danych i sposobu oceniania systemu rekomendującego

W badanych przez nas systemach rekomendujących wykorzystywane są dane w formie krotek <i>(numer użytkownika, numer filmu, ocena)</i>. Dane te mogą być przedstawione w formie macierzy użyteczności $U \in \mathbf{M}_{m \times n}$, gdzie $m$ oznacza liczbę użytkowników, $n$ liczbę filmów, zaś $U_{i,j}$ to ocena, jaką użytkownik $i$ wystawił filmowi $j$. Oceny są ze zbioru $\{1, 1.5, \dots 4.5, 5\}$, ale będziemy dopuszczali, by systemy rekomendujące przewidywały oceny także spoza tego zbioru (na przykład z odcinka $[1,5]$). $U_{ij} = 0$ oznacza, że użytkownik $i$ nie ocenił filmu $j$.

Aby zbadać skuteczność danego systemu rekomendującego, postępujemy następująco: wybieramy ustalonej wielkości próbkę niezerowych elementów $U$ i przesłaniamy je, to jest zamieniamy na zera. Następnie uczymy system na tak zmienionej macierzy użyteczności i porównujemy zaproponowane przez niego oceny próbki z ich rzeczywistymi wartościami, licząc błąd średniokwadratowy.

## Collaborative Filtering: podobieństwo kosinusowe

Zaimplementowaliśmy system typu <i>user-user</i> oparty na podobieństwie kosinusowym. Mając macierz użyteczności $U \in \mathbf{M}_{m \times n}$, chcemy przewidzieć oceny użytkownika $i$. W tym celu możemy znaleźć użytkowników podobnych do $i$ i uśrednić ich oceny. Potrzebujemy więc określić pewną miarę podobieństwa między użytkownikami.

Użytkownik $i$ jest reprezentowany przez jego wektor ocen $U_i \in \mathbb{R}^n$. Przez podobieństwo użytkowników będziemy rozumieć podobieństwo kosinusowe ich wektorów:

<center> $\rho(i,j) = \frac{U_i^TU_j}{||U_i||||U_j||}$.


Wadą tego podejścia jest konieczność obliczenia podobieństwa danego użytkownika ze wszystkimi pozostałymi, co jest czasochłonne. Okazuje się jednak, że w przypadku naszego zbioru danych system oparty na podobieństwie kosinusowym dość dobrze przewiduje oceny użytkowników.

## Singular Values Decomposition

Można pokazać, że każda macierz rzeczywista $U \in \mathbf{M}_{m \times n}$ ma rozkład według swoich wartości osobliwych (ang. <i>Singular Values Decomposition</i>). Dokładniej, istnieją macierze $P \in \mathbf{M}_{m \times m}, \Sigma \in \mathbf{M}_{m \times n}, Q \in \mathbf{M}_{n \times n}$ takie, że

<center>$U = P \Sigma Q^T$

oraz $P$ i $Q$ są ortogonalne, a $\Sigma$ to macierz diagonalna mająca na przekątnej wartości osobliwe $U$, to znaczy wartości własne $U^TU$.

Rozkład ten może być wykorzystany w systemie rekomendującym. Mając macierz użyteczności $U$, możemy znaleźć $P$ i $Q$ jak wyżej, interpretując wiersze $P$ jako wektory cech użytkowników, zaś wiersze $Q$ jako wektory cech produktów.

W pakiecie $\mathtt{scipy}$ dostępna jest funkcja $\mathtt{svds}$, która wyznacza $k$ największych wartości osobliwych danej macierzy oraz odpowiadające im wektory -- $k$ kolumn macierzy $P$ i $Q$. Jeśli największych $k$ wartości osobliwych $U$ jest znacznie większych od pozostałych wartości osobliwych, iloczyn tak "obciętych" macierzy $P, \Sigma, Q$ jest dość dobrym przybliżeniem $U$.


Pierwsze podejście polega na obliczeniu tak zdefiniowanego SVD. Nie jest ono jednak zbyt skuteczne, co nie dziwi, gdy zauważymy, że macierz $U$ jest zazwyczaj rzadka i traktujemy ją jako niepełną. $U_{ij} = 0$ nie oznacza, że użytkownik $i$ ocenił nisko produkt $j$, tylko że jego ocena jest nam nieznana; naszym celem jest ją przewidzieć. Możemy więc myśleć, że istnieje macierz $\tilde{U}$, której wszystkie elementy są niezerowe, reprezentująca oceny, jaki wszyscy użytkownicy wystawiliby wszystkim przedmiotom, a $U$ jest pewnym "podzbiorem" $\tilde{U}$. Tak naprawdę więc nie chcemy znaleźć rozkładu macierzy $U$, lecz rozkład $\tilde{U}$.

Zamiast rozkładać $U$ klasycznym algorytmem SVD, będziemy szukali takich macierzy $P, \Sigma, Q$, że $U_{ij} \approx (P\Sigma Q^T)_{ij}$ dla $i,j$ takich, że $U_{ij} \neq 0$. Podejście to możemy dalej uogólnić, porzucając założenia o ortogonalności $P, Q$, a także pozbywając się macierzy $\Sigma$. Zagadnienie, które będziemy w konsekwencji rozważać, brzmi następująco: dla danej macierzy $U \in \mathbf{M}_{m \times n}$ i parametru $k$ znaleźć $P \in \mathbf{M}_{m \times t}, Q \in \mathbf{M}_{t \times n}$ takie, że błąd średniokwadratowy

<center>$\mathbf{E} = \sum_{i,j : U_{ij} \neq 0} (U_{ij} - (PQ^T)_{ij})^2$

jest jak najmniejszy.

Zagadnienie to można rozwiązać metodą gradientu.

Do tej pory nie wymagaliśmy, by oceny przewidziane przez system były z zakresu $[1,5]$. Możemy to uzyskać, nakładając na $P_i^TQ_j$ funkcję sigmoid, przyjmującą wartości w przedziale $[0,1]$:
<center> $\sigma(t) = \frac{1}{1 + \textrm{e}^{-t}}$ </center>
i odpowiednio skalując wynik, to jest obliczając
<center> $\textrm{pred}(i,j) = 4 \sigma(P_i^TQ_j) + 1$ </center>
i minimalizując funkcję błędu
<center>$\mathbf{E} = \sum_{i,j : U_{ij} \neq 0} (U_{ij} - \textrm{pred}(i,j))^2$.
   

## Probabilistic Matrix Factorization

Podobnie jak poprzednio, będziemy szukać macierzy $P, Q$ takich, że $U = PQ^T$. Tym razem jednak będziemy zakładać, że $U$ pochodzi z modelu z Gaussowskim szumem.

Niech $\mathcal{N}(\cdot | \mu, \sigma^2)$ oznacza gęstość rozkładu normalnego o średniej $\mu$ i wariancji $\sigma^2$. Niech $I_{ij}$ będzie indykatorem równym 1, gdy $U_{ij} \neq 0$. Definiujemy rozkład warunkowy obserwacji:

<center>$p(U | P,Q, \sigma^2) = \prod_{i=1}^m \prod_{j=1}^n (\mathcal{N}(U_{ij} | P^T_iQ_j, \sigma^2))^{I_{ij}} $.

Będziemy również zakładać, że wiersze $P$ i $Q$, czyli wektory cech użytkowników i filmów, są niezależne i pochodzą z rozkładów normalnych o średnich 0 i macierzach kowariancji $\sigma_P^2\mathbf{I}, \sigma_Q^2\mathbf{I}$:

<center>$p(P | \sigma_P^2) = \prod_{i=1}^m \mathcal{N}(P_i | 0, \sigma_P^2\mathbf{I}), \quad p(Q|\sigma_Q^2) = \prod_{i=1}^n \mathcal{N}(Q_i | 0, \sigma_Q^2\mathbf{I}) $ .
    
Wówczas logarytm gęstości rozkładu a posteriori ma postać

<center> $\log p(P,Q | U, \sigma^2, \sigma_P^2, \sigma_Q^2) = -\frac{1}{2\sigma^2} \sum_{i=1}^m \sum_{j=1}^n I_{ij}(U_{ij} - P_i^TQ_j)^2 - \frac{1}{2\sigma_P^2} \sum_{i=1}^m P_i^T P_i - \frac{1}{2\sigma_Q^2} \sum_{i=1}^n Q_i^TQ_i - \frac{1}{2} \left(\left(\sum_{i=1}^m\sum_{j=1}^n I_{ij} \right) \log\sigma^2 + mk \log\sigma_P^2 + nk\log\sigma_Q^2 \right) + C$

dla pewnej stałej $C$ niezależnej od parametrów.

Przy ustalonych parametrach $\sigma, \sigma_P, \sigma_Q$ maksymalizacja tej funkcji jest równoważna minimalizacji funkcji błędu

<center> $\mathbf{E} = \sum_{i=1}^m \sum_{j=1}^n I_{ij}(U_ij - P_i^TQ_j)^2 + \lambda_P \sum_{i=1}^m || P_i ||^2 + \lambda_Q \sum_{i=1}^n || Q_i ||^2 $
    
dla $\quad \lambda_P = \sigma^2/\sigma^2_P,\quad \lambda_Q = \sigma^2/\sigma^2_Q$
    
Zauważmy, że zagadnienie to można rozumieć jako SVD z dodatkową regularyzacją. 